NameError: name 'scrapedate' is not defined

In [3]:
import requests
import json
import pickle
import os
import numpy as np
import time
import shutil
import datetime
import cv2
import urllib.request
import sqlite3 as sql
import base64
import ffmpy3
from file_read_backwards import FileReadBackwards

In [28]:
## Featurizer featurizes methods that convert json objects of the appropriate type into features

class Featurizer:
    def __init__(self):
        pass
        #self.name = name

    # takes in text pickle and scrapedate, returns vector of 14 elements
    # the first element is count of texts sent in the 24 hours before the scrape
    # the last element is count of texts sent on the 24 hour window 14 days before the scrapedate
    def textFreqVec14(self, text, scrapedate):
        
        textFreqVec = np.zeros((14,))
        
        mseconds_in_twoweeks = 1209600000;
        mseconds_in_day = 86400000;
        
        # assuming unordered texts (WHICH TURNS OUT IS THE CASE)
        for day in range(0,14):
            time_during_week_ub = scrapedate - ((day)*mseconds_in_day)    
            time_during_week_lb = scrapedate - ((day+1)*mseconds_in_day)
            for t in range(0,len(text)):
                text_date = int(json.loads(text[t])['date'].encode('ascii','ignore'))
                if ((time_during_week_ub > text_date) and (text_date > time_during_week_lb)):
                    textFreqVec[day] += 1.0
                    
        return textFreqVec
    

    # takes in call pickle and scrapedate, returns vector of 14 elements
    # the first element is count of calls sent in the 24 hours before the scrape
    # the last element is count of calls sent on the 24 hour window 14 days before the scrapedate
    def callFreqVec14(self, call, scrapedate):
        
        callFreqVec = np.zeros((14,))
        
        mseconds_in_twoweeks = 1209600000;
        mseconds_in_day = 86400000;
        
        for day in range(0,14):
            time_during_week_ub = scrapedate - ((day)*mseconds_in_day)    
            time_during_week_lb = scrapedate - ((day+1)*mseconds_in_day)
            for c in range(0,len(call)):
                call_date = int(json.loads(call[c])['date'].encode('ascii','ignore'))
                if ((time_during_week_ub > call_date) and (call_date > time_during_week_lb)):
                    callFreqVec[day] += 1.0
                    
        return callFreqVec
    
    
    # input is tweets pickle, return master vector
    def embeddingToMastersum(self, tweets):
        
        masterSum = np.zeros((300,))

        # add every word vector into master sum
        for i in range(0,len(tweets)):
            try:
                masterSum += self.tweetToEmbedding(tweets[i])
            except:
                pass
        
        return masterSum

    
    # input is one single tweet, returns vector embedding of entire tweet.
    # eg: responseobject.json()[0]
    def tweetToEmbedding(self, tweet):

        q = tweet['text'].split()

        sumVector = np.zeros((300,))

        # turn every word into embedding for 1 tweet, add all vectors
        for i in range(0,len(q)):
            try:
                sumVector += model[q[i]]
            except KeyError:
                pass
            except IndexError:
                pass

        return sumVector
    
    
    # input is tweets pickle, returns follow count
    def followerCount(self, tweets):

        followerCount = 0

        # get
        try:
            followerCount = json.loads(tweets[0])['user']['followers_count']
        except KeyError:
            pass
        except IndexError:
            pass

        return followerCount

    # input is tweets pickle, returns friend count
    def followingCount(self, tweets):

        followingCount = 0

        try:
            followingCount = json.loads(tweets[0])['user']['friends_count']
        except KeyError:
            pass
        except IndexError:
            pass

        return followingCount

    # input is tweets pickle, return avg likes per post for the last 2 weeks
    def twitterLikeFreq(self, tweets, scrapedate):

        twitLikeVec = np.zeros((1,))
        
        mseconds_in_twoweeks = 1209600000;
        mseconds_in_day = 86400000;
        
        # upper and lower bounds
        time_during_week_ub = scrapedate - ((0)*mseconds_in_day)    
        time_during_week_lb = scrapedate - ((14)*mseconds_in_day)
        for t in range(0,len(tweets)):

            utc = json.loads(tweets[t])['created_at']

            tweet_date = int(time.mktime(time.strptime(utc,"%a %b %d %H:%M:%S +0000 %Y"))) * 1000

            if ((time_during_week_ub > tweet_date) and (tweet_date > time_during_week_lb)):
                twitLikeVec[0] += 1.0
                    
        return twitLikeVec/14


    # input is tweets pickle, return avg retweets per post for the last 2 weeks
    def twitterRetweetFreq(self, tweets, scrapedate):

        twitRTVec = np.zeros((1,))
        
        mseconds_in_twoweeks = 1209600000;
        mseconds_in_day = 86400000;
        
        # upper and lower bounds
        time_during_week_ub = scrapedate - ((0)*mseconds_in_day)    
        time_during_week_lb = scrapedate - ((14)*mseconds_in_day)
        for t in range(0,len(tweets)):

            utc = json.loads(tweets[t])['created_at']

            tweet_date = int(time.mktime(time.strptime(utc,"%a %b %d %H:%M:%S +0000 %Y"))) * 1000

            if ((time_during_week_ub > tweet_date) and (tweet_date > time_during_week_lb)):
                twitRTVec[0] += json.loads(tweets[i])['favorite_count']
                    
        return twitRTVec/14
    
    # input is contacts pickle, returns number of contacts
    def numOfContacts(self, contacts):
        return len(contacts)
    
    # input is instagram pickle, return two features: follows count, followed by count  
    def instagramThings(self, instagram):
        
        followsFollowed = np.zeros((2,))
        
        try:
            if(type(json.loads(instagram[0])) == str):
                print("DAMN YOU DAMON!!(ig)")
            else: # its a dictionary like it's supposed to be
                followsFollowed[0] = json.loads(instagram[0])['data']['counts']['follows']
                followsFollowed[1] = json.loads(instagram[0])['data']['counts']['followed_by']
        except IndexError:
            pass
        
        return followsFollowed
    
    # takes in instagramMedia pickle scrape date, spits out filter usage frequency for the past 2 weeks
    def instagramFilterFreq(self, instagramMedia, scrapedate):
        
        instaFiltVec = np.zeros((1,))
        
        mseconds_in_twoweeks = 1209600000;
        mseconds_in_day = 86400000;
        
        # upper and lower bounds
        time_during_week_ub = scrapedate - ((0)*mseconds_in_day)    
        time_during_week_lb = scrapedate - ((14)*mseconds_in_day)
        for t in range(0,len(instagramMedia)):

            utc = json.loads(instagramMedia[t])['created_at']
            
            if(this != '[object Object]'):
                    igpost_date = int(json.loads(this)['created_time']) * 1000
            else:
                print("DAMN YOU DAMON!!(ig)")

            if ((time_during_week_ub > igpost_date) and (igpost_date > time_during_week_lb)):
                instaFiltVec[0] += 1.0
                    
       
        nofilter_count = 0

        for i in range(0,instaFiltVec[0]):
            if(json.loads(tweets[i])['filter'] == Normal):
                nofilter_count += 1
        
        instaFiltVec[0] -= nofilter_count
        
        return instaFiltVec
        
       
    
    # takes in instagramMedia pickle and scrape date, returns a vector that
    # contains a normalized percentage (0-1) for the usage of the filters
    # listed below for the past 2 weeks: 
    # Valencia, X-Pro II, Hefe, Amaro, Rise, Willow, Crema, Inkwell
    
    # output example: [0.5,0,0,0,0,0.5,0,0] 
    # interpretation: user used valencia half the time, Willow the other half
    # of time, for the past 2 weeks of Instagram posts.1`sas
    def instagramFilterVector(self, instagramMedia, scrapedate):
        
        filters = {'Valencia':0,'X-Pro II':0, 'Hefe':0, 'Amaro':0, 'Rise':0, 'Willow':0, 'Crema':0, 'Inkwell':0}
        filtervec = np.zeros((8,))
        
        numposts = 0
        
        mseconds_in_twoweeks = 1209600000;
        mseconds_in_day = 86400000;
        
        # upper and lower bounds
        time_during_week_ub = scrapedate - ((0)*mseconds_in_day)    
        time_during_week_lb = scrapedate - ((14)*mseconds_in_day)
        for t in range(0,len(tweets)):

            utc = json.loads(tweets[t])['created_at']
            
            if(this != '[object Object]'):
                igpost_date = int(json.loads(this)['created_time']) * 1000
            else:
                print("DAMN YOU DAMON!!(ig)")

            if ((time_during_week_ub > igpost_date) and (igpost_date > time_during_week_lb)):
                numposts += 1.0
                
                
        for i in range(0,numposts):
            filt = json.loads(instagramMedia[i])['filter']
            if(filt in filters):
                filters[filt] += 1
            else:
                pass
            
        for i in range(0,8):
            filtervec[i] = filters[list(filters)[i]]

        # percentage of how much the filters are used as a normalized vector
        return filtervec/(numposts)
    
    # takes in InstagramMedia, returns comment and like frequency for the 
    # past 2 weeks.
    def instagramLikeComFreq(self, instagramMedia, scrapedate):
        
        counts = np.zeros((2,))
        
        postcount = 0
        
        mseconds_in_twoweeks = 1209600000;
        mseconds_in_day = 86400000;
        
        # upper and lower bounds
        time_during_week_ub = scrapedate - ((0)*mseconds_in_day)    
        time_during_week_lb = scrapedate - ((14)*mseconds_in_day)
        for t in range(0,len(instagramMedia)):

            utc = json.loads(instagramMedia[t])['created_at']
            
            if(this != '[object Object]'):
                    igpost_date = int(json.loads(this)['created_time']) * 1000
            else:
                print("DAMN YOU DAMON!!(ig)")

            if ((time_during_week_ub > igpost_date) and (igpost_date > time_during_week_lb)):
                postcount += 1.0
        
        for i in range(0,postcount):
            counts[0] += json.loads(instagramMedia[i])['likes']['count']
            counts[1] += json.loads(instagramMedia[i])['comments']['count']
        
        if (counts[0] + counts[1] == 0):
            return np.zeros((2,))
        else:
            # likes per post for past 2 weeks
            return counts/postcount


    # takes in instagramMedia pickle and scrapedate, returns IG post 
    # frequency for the past 2 weeks
    def instagramPostFreq(self, instagramMedia, scrapedate):
        
        postcount = np.zeros((1,))
        
        mseconds_in_twoweeks = 1209600000;
        mseconds_in_day = 86400000;
        
        # upper and lower bounds
        time_during_week_ub = scrapedate - ((0)*mseconds_in_day)    
        time_during_week_lb = scrapedate - ((14)*mseconds_in_day)
        for t in range(0,len(instagramMedia)):

            utc = json.loads(instagramMedia[t])['created_at']
            
            if(this != '[object Object]'):
                    igpost_date = int(json.loads(this)['created_time']) * 1000
            else:
                print("DAMN YOU DAMON!!(ig)")

            if ((time_during_week_ub > igpost_date) and (igpost_date > time_during_week_lb)):
                postcount[0] += 1.0
                
        return postcount/14
        
        
    # takes instagramMedia, returns pixelwise average values for [H,S,V]
    # (Hue, Saturation, Value) for all posts in the past 2 weeks, the 
    # count of faces as a frequency of faces per picture, for the past
    # 2 weeks as well.
    
    # testvariable:
    # empty string: "" if not testing
    # 
    def averageHSVF(self, instagramMedia, scrapedate):


        seconds_intwo_weeks = 1209600;

        ## ms into secs
        scrapedate = int(str(scrapedate)[:-3]) 

        two_weeks_prior = scrapedate - seconds_intwo_weeks

        igpost_date = 0
        saved_index = 0

        a = instagramMedia

        # find the index of the oldest call that is not older than the date "two weeks prior"
        for i in range(0,len(a)):
            try:
                this = a[len(a)-(i+1)]
                if(this != '[object Object]'):
                    igpost_date = int(json.loads(this)['created_time'])
                else:
                    print("DAMN YOU DAMON!!(ig)")
            except KeyError:
                pass
            except IndexError:
                pass

            if (two_weeks_prior < igpost_date):
                saved_index = len(a) - (i+1)
                break


        

        if(os.path.exists("./ILLSTOPBLINKINGSOON")):
            shutil.rmtree('./ILLSTOPBLINKINGSOON')

        os.mkdir('./ILLSTOPBLINKINGSOON')

        avgs = np.zeros((4,))

        ## to avoid division by 0
        if(saved_index == 0):
            return avgs



        for i in range(0,saved_index):
            url = json.loads(instagramMedia[i])['images']['thumbnail']['url']
            urllib.request.urlretrieve(url, './ILLSTOPBLINKINGSOON/' + str(i) + '.jpg')


        # face_cascade here is a pre trained classifier for frontal faces 
        face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

            
        avgHue = 0
        avgSatur = 0
        avgVal = 0
        avgFaces = 0

        ## GODS OF PROGRAMMING, FORGIVE ME FOR THIS TRIPLE NEST

        for k in range(0, saved_index):
            ## BGR and not RGB because imread reads in BGR
            img = cv2.imread('./ILLSTOPBLINKINGSOON/' + str(k) + '.jpg')
            
            hsv = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
            
            grayImage = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(grayImage,  scaleFactor=1.1, minNeighbors=5, flags = cv2.CASCADE_SCALE_IMAGE)
            
            avgFaces += len(faces)
            
            for i in range(0,hsv.shape[0]):
                for j in range(0,hsv.shape[1]):
                    avgHue += hsv[i,j,0]
                    avgSatur += hsv[i,j,1]
                    avgVal += hsv[i,j,2]
                    
        

        sums = [avgHue,avgSatur,avgVal]

        ## 22500 = 150x150 = instagram photo thumbnail shape
        avgs = list(map(lambda x: x/(22500*saved_index), sums)).append(avgFaces/saved_index)

        shutil.rmtree('./ILLSTOPBLINKINGSOON')

        return avgs      

    
    #input is texts pickle, return master vector 
    def embeddingToMastersumText(self, texts):
        
        masterSum = np.zeros((300,))

        # add every word vector into master sum
        for i in range(0,len(texts)):
            try:
                masterSum += self.textToEmbedding(texts[i])
            except:
                pass
        
        return masterSum

    
    # input is single text, returns vector embedding of entire text.
    def textToEmbedding(self, text):
        
        q = json.loads(text)["body"].split()

        sumVector = np.zeros((300,))

        # turn every word into embedding for 1 tweet, add all vectors
        for i in range(0,len(q)):
            try:
                sumVector += model[q[i]]
            except KeyError:
                pass
            except IndexError:
                pass

        return sumVector
    
    
    # returns [q1,q2,q3,q4,q5,q6,q7,q8,q9] and sum of all these scores
    def labelGenerator(self, phq):
        
        labelVector = np.zeros((10,))
        sumOfScores = 0
        
        # print(phq)
        
        for i in range(0,9):
            temp = int(json.loads(phq[0])['Q' + str(i)])
            labelVector[i] = temp
            labelVector[9] += temp
            
        return labelVector

    
        import base64
        import ffmpy3

        bytestream = base64.b64decode(voice[0])

        fh = open("test.3gp","wb")
        fh.write(bytestream)
        fh.close()

        ff = ffmpy3.FFmpeg( inputs={'test.3gp': None}, outputs={'test1.wav': None})
        ff.run()

        from file_read_backwards import FileReadBackwards

        with FileReadBackwards("out.csv", encoding="utf-8") as frb:
            for l in frb:
                #print(l)
                break

        b = l[9:].split(',')
        a = list(map(float, b))

        
    def voiceFeaturizer(self, voice):

        audiofeaturevec = np.zeros((1583,))

        if(len(voice) == 0):
            return audiofeaturevec

        # base64 string -> bitstring -> bitstream -> write into 3gp file
        bytestream = base64.b64decode(voice[0])
        fh = open("audio.3gp","wb")
        fh.write(bytestream)
        fh.close()

        # wav -> 3gp
        ff = ffmpy3.FFmpeg( inputs={'audio.3gp': None}, outputs={'audio.wav': None})
        ff.run()

        os.remove("audio.3gp")

        # call to openSMILE
        os.system( os.getcwd() + '/openSMILE-2.1.0/bin/linux_x64_standalone_static/SMILExtract -C ' + os.getcwd() + '/openSMILE-2.1.0/config/emobase2010.conf -I audio.wav -O "out.csv"')

        os.remove("audio.wav")

        # csv file has a giant header. last line contains the features we want
        # so we read the last line, cut out more useless string with 'l[9:]
        with FileReadBackwards("out.csv", encoding="utf-8") as frb:
            for l in frb:
                b = l[9:].split(',')
                break

        os.remove("out.csv")

        # make list of string into list of floats
        a = list(map(float, b))

        # make sklearn happy
        for i in range(0,len(a)):
            audiofeaturevec[i] = a[i]

        return audiofeaturevec

In [7]:
import unittest
import os
import pickle
import numpy as np
import json

# f = Featurizer()

b1 = pickle.load(open( "datafor16:13" + "/DP" + "8619" +  "Instagram media" + ".p", "rb" ))        
# for i in range(0,len(b1)):
#     print(int(json.loads(b1[i])['date'].encode('ascii','ignore')))

igpost_date = int(json.loads(b1[0])['created_time']) * 1000
igpost_date

1514323274000

In [4]:
b1 = pickle.load(open( "datafor16:13" + "/DP" + "8619" +  "audio" + ".p", "rb" ))        

len(b1)

1

In [206]:
import base64
import ffmpy3
import os
from file_read_backwards import FileReadBackwards

# base64 string -> bitstring -> bitstream -> write into 3gp file
bytestream = base64.b64decode(voice[0])

NameError: name 'voice' is not defined

In [74]:
class TestFeaturizer(unittest.TestCase):
    
    if __name__ == '__main__':
        unittest.main(argv=['first-arg-is-ignored'], exit=False)
    
    def test_labelGenerator(self):
        b1 = pickle.load(open( "datafor16:13" + "/DP" + "7276" +  "phq" + ".p", "rb" ))
        self.assertSequenceEqual(f.labelGenerator(b1).tolist(),[3,1,0,2,0,0,0,0,0,6])
        
    def test_textFreqVec14(self):
        b1 = pickle.load(open( "datafor16:13" + "/DP" + "0660" +  "text" + ".p", "rb" ))
        self.assertSequenceEqual(f.textFreqVec14(b1,1515042722290),[2, 3, 0, 0, 2, 5, 4, 2, 2, 2, 1, 1, 2, 0])
        
    def test_callFreqVec14(self):
        b1 = pickle.load(open( "datafor16:13" + "/DP" + "5904" +  "log" + ".p", "rb" ))
        self.assertSequenceEqual(f.callFreqVec14(b1,1515042722290),[23, 34, 16, 6, 8, 26, 0, 0, 0, 0, 0, 0, 0, 0])    
        
    def test_dailyTextFre(self):
        self.assertEqual(4,4)
        
    def add(self,x,y):
        return x + y
        
    def test_errorexample(self):
        self.assertRaises(TypeError,self.add,"3",4)    

/home/vape/.local/lib/python3.5/site-packages/ipykernel_launcher.py:15: ResourceWarning: unclosed file <_io.BufferedReader name='datafor16:13/DP5904log.p'>
  from ipykernel import kernelapp as app
E../home/vape/.local/lib/python3.5/site-packages/ipykernel_launcher.py:7: ResourceWarning: unclosed file <_io.BufferedReader name='datafor16:13/DP7276phq.p'>
  import sys
./home/vape/.local/lib/python3.5/site-packages/ipykernel_launcher.py:11: ResourceWarning: unclosed file <_io.BufferedReader name='datafor16:13/DP0660text.p'>
  # This is added back by InteractiveShellApp.init_path()
E
ERROR: test_callFreqVec14 (__main__.TestFeaturizer)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-72-5a4136992325>", line 16, in test_callFreqVec14
    self.assertSequenceEqual(f.callFreqVec14(b1,1515042722290),[23, 34, 16, 6, 8, 26, 0, 0, 0, 0, 0, 0, 0, 0])
  File "/usr/lib/python3.5/unittest/case.py", line 944, in assertSequenc

In [322]:
b1 = pickle.load(open( "datafor16:13" + "/DP" + "8619" +  "Instagram media" + ".p", "rb" ))
len(b1)

igpost_date = int(json.loads(b1[0])['created_time'])
igpost_date

1514323274

In [220]:
## PREPROCESSING

from sklearn.preprocessing import Imputer

# replace missing values with mean of their corresponding features
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
mtr = imp.fit_transform(mtr)

# normalize (features now have gauss dist., 0 mean and unit variance)
mtr = sklearn.preprocessing.scale(mtr)

# all of our data
bigBadMatrix = g.featureMatrix

# shuffle row-wise
np.random.shuffle(bigBadMatrix)




# DATA SPLIT (#nosnooping)

# TEST DATA (%15 percent of data)
numofppl_index = ftrMtrx.shape[0] - 1
cut_index = int(ftrMtrx.shape[0] * 0.85)

test_data = bigBadMatrix[cut_index:numofppl,:]

# TRAINING AND TEST DATA (%85 percent of data)

train_data = bigBadMatrix[0:cut_index,:]









## hyper parameter optimization

from sklearn.grid_search import GridSearchCV







c_range = list(range(1, 30))
parameters = {'kernel':('linear', 'rbf', 'poly', 'sigmoid', 'precomputed'), 'C':c_range}
parameters['kernel']

from sklearn.grid_search import GridSearchCV
from sklearn import svm



np.random.shuffle



svc = svm.SVC()
grid = GridSearchCV(svc, parameters, cv=2, scoring='accuracy')

clf.fit()


('linear', 'rbf', 'poly', 'sigmoid', 'precomputed')

In [17]:
b1 = pickle.load(open( "datafor16:13" + "/DP" + "8619" +  "Instagram media" + ".p", "rb" ))

filt = json.loads(b1[0])['filter']
filt

'Normal'

In [23]:
x = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]], np.int32)
x

array([[ 1,  2,  3],
       [ 4,  5,  6],
       [ 7,  8,  9],
       [10, 11, 12]], dtype=int32)

In [24]:
x[:,1:2] # subcolumn

array([[ 2],
       [ 5],
       [ 8],
       [11]], dtype=int32)

In [292]:
x[0:2,:] #subrow 

array([[1, 2, 3],
       [4, 5, 6]], dtype=int32)

In [21]:
np.random.shuffle(x)
x

array([[ 7,  8,  9],
       [ 1,  2,  3],
       [10, 11, 12],
       [ 4,  5,  6]], dtype=int32)

In [26]:
labelVector = np.zeros((9,))
sumOfScores = 0

# print(phq)
a = {"Q0":"3","Q1":"3","Q2":"3","Q3":"3","Q4":"3","Q5":"2","Q6":"3","Q7":"3","Q8":"1"}
for i in range(0,8):
    temp = int(a['Q' + str(i)])
    labelVector[i] = temp
    labelVector[9] += temp

labelVector

IndexError: index 9 is out of bounds for axis 0 with size 9

In [27]:
np.zeros((9,))


array([0., 0., 0., 0., 0., 0., 0., 0., 0.])